## Configure FACTS run

In [1]:
# Configure directories and experiment

factsdir = '/opt/facts' # FACTS root directory
exp_name = 'jupyter01' # desired experiment name

experiment_config = """
global-options:
    nsamps: 200
    scenario: ssp245
    pyear_start: 2020
    pyear_end: 2150
    pyear_step: 10
    baseyear: 2005
    pipeline_file: 'pipeline.global.yml'

climate_step:
    temperature:
        module_set: "fair"
        module: "temperature"
        generates_climate_output: true

sealevel_step:
    GrIS1f:
        module_set: "FittedISMIP"
        module: "GrIS"
        include_in_workflow:
            - "wf1f"
            - "wf2f"
            - "wf3f"

    deconto21:
        module_set: "deconto21"
        module: "AIS"
        include_in_workflow:
            - "wf3f"

    bamber19:
        module_set: "bamber19"
        module: "icesheets"
        include_in_workflow:
            - "wf4"

    larmip:
        module_set: "larmip"
        module: "AIS"
        include_in_workflow:
            - "wf2f"

    ar5glaciers:
        module_set: "ipccar5"
        module: "glaciers"
        options:
            gmip: 2
        include_in_workflow:
            - "wf1f"
            - "wf2f"
            - "wf3f"
            - "wf4"

    ar5AIS:
        module_set: "ipccar5"
        module: "icesheets"
        pipeline_file: "pipeline.AIS.global.yml"
        include_in_workflow:
            - "wf1f"

    ocean:
        module_set: "tlm"
        module: "sterodynamics"
        include_in_workflow:
            - "wf1f"
            - "wf2f"
            - "wf3f"
            - "wf4"

    lws:
        module_set: "ssp"
        module: "landwaterstorage"
        options:
            scenario: "ssp2"
            dcrate_lo: -0.4
        include_in_workflow:
            - "wf1f"
            - "wf2f"
            - "wf3f"
            - "wf4"


totaling_step:
    total:
        module_set: "facts"
        module: "total"
        loop_over_workflows: true
        loop_over_scales: true
        stages:
            - workflow

"""

Write the experiment `config.yml` file. 

In [2]:
import os 
import sys

os.chdir(factsdir)
import runFACTS

exp_dir=os.path.join(factsdir,'experiments',exp_name)
resourcedir=os.path.join(factsdir,'resources')

try:
    os.makedirs(exp_dir)
except:
    None

f = open(os.path.join(exp_dir,'config.yml'),'w')
f.write(experiment_config)
f.close()


## Run FACTS

Run the newly created FACTS experiment.

In [ ]:
runFACTS.run_experiment(exp_dir,resourcedir=resourcedir)

EnTK session: re.session.3681b5d6-986f-11ee-8261-0242ac110002
Creating AppManager
Setting up ZMQ queues                                                         ok
AppManager initialized                                                        ok
Validating and assigning resource manager                                     ok
Setting up ZMQ queues                                                        n/a
new session: [re.session.3681b5d6-986f-11ee-8261-0242ac110002]                 \
zmq proxy  : [tcp://172.17.0.2:10008]

****** STEP: climate_step ******


                                         ok
create pilot manager                                                          ok
submit 1 pilot(s)
        pilot.0000   local.localhost           2 cores       0 gpus           ok


## Visualize output

In [ ]:
os.listdir(os.path.join(exp_dir,'output'))

In [ ]:
import xarray as xr

datafile='jupyter01.total.workflow.wf4.global.nc'
ds = xr.open_dataset(os.path.join(exp_dir,'output',datafile))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(ds.years,ds.sea_level_change.quantile(.5,dim='samples'))
plt.plot(ds.years,ds.sea_level_change.quantile(.17,dim='samples'))
plt.plot(ds.years,ds.sea_level_change.quantile(.83,dim='samples'))

plt.ylabel('mm')
plt.title(datafile)

In [ ]:
dq=ds.sea_level_change.quantile([.17,.5,.83],dim='samples')
u=dq[:,:,0]
print(u.to_pandas())